In [0]:
import tensorflow as tf

In [0]:
layer_in_block = {'vgg11': [1, 1, 2, 2, 2],
                  'vgg13': [2, 2, 2, 2, 2],
                  'vgg16': [2, 2, 3, 3, 3],
                  'vgg19': [2, 2, 4, 4, 4]}

class SEBlock(tf.keras.layers.Layer):
    def __init__(self, input_channels, r=16):
        super(SEBlock, self).__init__()
        self.pool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc1 = tf.keras.layers.Dense(units=input_channels // r)
        self.fc2 = tf.keras.layers.Dense(units=input_channels)

    def call(self, inputs, **kwargs):
        branch = self.pool(inputs)
        branch = self.fc1(branch)
        branch = tf.nn.relu(branch)
        branch = self.fc2(branch)
        branch = tf.nn.sigmoid(branch)
        branch = tf.expand_dims(input=branch, axis=1)
        branch = tf.expand_dims(input=branch, axis=1)
        output = tf.keras.layers.multiply(inputs=[inputs, branch])
        return output


class VggConv(tf.keras.layers.Layer):
  def __init__(self, filter_num=None, kernel_size=(3, 3), \
               activation='relu', padding='same', kernel_initializer='he_normal'):
    super(VggConv, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                        kernel_size=kernel_size,
                                        activation=activation,
                                        padding=padding,
                                        kernel_initializer=kernel_initializer)   
    
  def call(self, inputs, training=None):
    x = self.conv1(inputs)
    return x

class VggConvs(tf.keras.layers.Layer):
  def __init__(self, pool_size=(2, 2)):
    super(VggConvs, self).__init__()
    self.bn1 = tf.keras.layers.BatchNormalization()
    
    self.pool1 = tf.keras.layers.MaxPool2D(pool_size=pool_size)
  
  def call(self, inputs, training=None):
    x = self.bn1(inputs, training=training)
    x = self.pool1(x)
    return x

class VggDense(tf.keras.layers.Layer):
  def __init__(self, filter_num=None, classes=1):
    super(VggDense, self).__init__()
    self.Flatten = tf.keras.layers.Flatten()
    self.Dense1 = tf.keras.layers.Dense(filter_num, activation='relu')
    self.bn1 = tf.keras.layers.BatchNormalization()
    if classes == 1:
      self.Dense2 = tf.keras.layers.Dense(classes, activation=tf.keras.activations.sigmoid)
    else:
      self.Dense2 = tf.keras.layers.Dense(classes, activation=tf.keras.activations.softmax)

  def call(self, inputs):
    x = self.Flatten(inputs)
    x = self.Dense1(x)
    x = self.bn1(x)
    x = self.Dense2(x)

    return x


def vgg_convs_layer(filter_num=None, blocks=None,  kernel_size=(3, 3), \
                    activation='relu', padding='same', kernel_initializer='he_normal', \
                    pool_size=(2, 2), use_se = False):
  vgg_block = tf.keras.Sequential()
  for i in range(blocks):
    vgg_block.add(VggConv(filter_num=filter_num,  kernel_size=kernel_size, activation=activation, padding=padding, kernel_initializer=kernel_initializer))
  vgg_block.add(VggConvs(pool_size=pool_size))
  if use_se == True:
    vgg_block.add(SEBlock(input_channels=filter_num))
  return vgg_block


class VggNet(tf.keras.Model):
  def __init__(self, layer='vgg16', use_se = False, classes=1):
    super(VggNet, self).__init__()
 
    self.conv1 = vgg_convs_layer(filter_num = 64, blocks = layer_in_block[layer][0], use_se = use_se)
    self.conv2 = vgg_convs_layer(filter_num = 128, blocks =  layer_in_block[layer][1], use_se = use_se)
    self.conv3 = vgg_convs_layer(filter_num = 256, blocks = layer_in_block[layer][2], use_se = use_se)
    self.conv4 = vgg_convs_layer(filter_num = 512, blocks = layer_in_block[layer][3], use_se = use_se)
    self.conv5 = vgg_convs_layer(filter_num = 512, blocks = layer_in_block[layer][4], use_se = use_se)
    self.dense = VggDense(filter_num = 256, classes = classes)

  def call(self, inputs, cam=None):
    x = self.conv1(inputs)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x_cam = self.conv5(x)
    x = self.dense(x_cam)

    if cam == 'grad':
      return x_cam, x
    
    return x

In [0]:
def vgg_11(classes):
  return VggNet(layer='vgg11', classes)

def vgg_13(classes):
  return VggNet(layer='vgg13', classes)

def vgg_16(classes):
  return VggNet(layer='vgg16', classes)

def vgg_19(classes):
  return VggNet(layer='vgg19', classes)

def se_vgg_16(classes):
  return VggNet(layer='vgg16', use_se = True, classes)

def se_vgg_19(classes):
  return VggNet(layer='vgg19', use_se = True, classes)